Program som genererer GIF-filer som viser utbredelsen av energibrønner i Norge fra ulike brønnborere gjennom flere år

In [75]:
#Importerer pakker
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import folium
import webbrowser
from folium.features import DivIcon
from html2image import Html2Image
import os
from PIL import Image
import glob

# Leser av Excel-fil som er lastet ned fra NGU Granada:
df = pd.read_excel("GrunnvannsBorehull_Oslo.xlsx", sheet_name="EnergiBrønn",usecols='E,F,G,X,Y')

# Gjør om datoer i excel-fil til ønsket datetime-format:
df = df.sort_values('boreDato')
datokolonne = 2
datoer = df.iloc[:,datokolonne]
pd.to_datetime(datoer)
pd.to_datetime(df.iloc[:,datokolonne])

7048   1900-01-01
7251   1900-01-01
5524   1900-01-01
2170   1900-01-01
1981   1900-01-01
          ...    
6657          NaT
7697          NaT
7849          NaT
7850          NaT
8657          NaT
Name: boreDato, Length: 9598, dtype: datetime64[ns]

Oppretter egne datasett for de 10 største oppdragstakerne

In [76]:
#Skiller ut de 10 største oppdragstakerne:
baasum = df[df['oppdragstaker'].str.contains('|'.join(['Båsum Boring A.S','Båsum Boring Trøndelag AS']))] 
rototec = df[df['oppdragstaker'].str.contains('|'.join(['ROTOTEC AS','Rototec AS']))]
kraftenergibronn = df[df['oppdragstaker'].str.contains('Kraft Energi og brønnboring AS')]
aarsleff = df[df['oppdragstaker'].str.contains('|'.join(['Sør-Norsk Brønnboring AS','SØR-NORSK BORING AS']))]
seabrokers = df[df['oppdragstaker'].str.contains('|'.join(['Østlandet brønn- og energiboring as','ØSTLANDET BRØNN OG ENERGIBORING AS','Østlandske brønn og energiboring AS',
                                                           'Østlandske brønnboring a/s','SEABROKERS GEOENERGI AS','Seabrokers Entreprenør Service as','Seabrokers Fundamentering AS']))]
vestnorsk = df[df['oppdragstaker'].str.contains('Vestnorsk Brunnboring As')]
universal = df[df['oppdragstaker'].str.contains('|'.join(['UNIVERSAL BRØNNBORING AS', 'Universal Brønnboring A/S']))]
follo = df[df['oppdragstaker'].str.contains('Follo brønnboring AS')]
vaeraas = df[df['oppdragstaker'].str.contains('Værås Brønnboring')]
brodrenemyhre = df[df['oppdragstaker'].str.contains('Brødrene Myhre A/S')]

enkelthusholdning = df[df['brønnBrukOmfang'].str.contains('Enkelthusholdning')]
storre_anlegg = df[df['brønnBrukOmfang'].str.contains('Større anlegg')]
andre_anlegg = df[df['brønnBrukOmfang'].str.contains('|'.join(['Gårdsbruk', 'Hytte/fritidsbolig', 'Turistnæring', 'Ukjent']))]

# Skriver ut totalt antall brønner for de 10 største oppdragstakerne:
print(len(df))                          #1992-2023 (alle)
print(len(baasum))                      #1996-2023
print(len(rototec))                     #2012-2023
print(len(kraftenergibronn))            #2005-2023
print(len(aarsleff))                    #2002-2023
print(len(seabrokers))                  #1996-2023
print(len(vestnorsk))                   #1997-2023
print(len(universal))                   #1996-2019
print(len(follo))                       #2001-2023
print(len(vaeraas))                     #1992-2023
print(len(brodrenemyhre))               #1993-2023

#Bruk denne til å finne oppdragstakerens første og siste år:
#print(brodrenemyre.sort_values('boreDato')[0:-1])

9598
2653
2537
601
209
357
0
881
123
128
218


Funksjon for å telle opp antall brønner boret hvert år av gitt arbeidstaker:

In [77]:
def bronner_hvert_aar_borer(borer,aar):
    #input:     borer:  Datasett med boredata for valgt oppdragstaker, bruk df for alle oppdragstakere
    #           aar:    Året hvor man vil telle opp antall brønner i
    #output:            Datasett med boredata for valgt oppdragstaker og valgt år

    borer = borer.sort_values('boreDato')
    datoer_borer = borer.iloc[:,datokolonne]
    datoer_aar = [date for date in borer.iloc[:,datokolonne] if date.year == aar]
    indeks_aar = []
    for i in range(0,len(datoer_aar)):
        hvor = np.where(datoer_borer == np.datetime64(datoer_aar[i]))
        indeks_aar = np.concatenate((indeks_aar,hvor[0]))
    if len(indeks_aar) >=1:
        bronner_aar_borer = borer.iloc[int(indeks_aar[0]):int(indeks_aar[-1])+1,:]
    else:
        bronner_aar_borer = pd.DataFrame()
    return bronner_aar_borer

Funksjon for å generere GIF for utbygging av energibrønner av gitt oppdragstaker gjennom flere år

In [82]:
def bronner_fra_oppdragstaker(borer,startaar,sluttaar,navn,figurnavn,sted):
    # input:     borer:      Datasett med boredata for valgt oppdragstaker, bruk df for alle oppdragstakere
    #            startaar:   Året hvor oppdragstakerens første brønn bores. Animasjonen vil starte året før
    #            sluttaar:   Siste år som animasjonen skal gå for, ofte årets årstall
    #            navn:       String med kort navn som brukes til å navngi mappe og filer
    #            figurnavn:  String med fullt navn på oppdragstaker, blir vist på figuren
    #            sted:       'Norge' eller 'Oslo'. Sted som velger plassering av kartet og grafikken
    # output:    (mappe):     En ny mappe som inneholder html- og png- filer for hvert år, samt den ferdige gif-filen. Merk at det ikke må eksistere en mappe med samme navn fra før.
    
    if sted == 'Norge':
        zoomlevel = 5
        koord_lat = [65, 71, 69.3, 57, 57, 57]
        koord_long = [18, 2, 2, 21.5, 31.6, 34.3]
    elif sted == 'Oslo':
        zoomlevel = 11
        koord_lat = [59.95, 60.10, 60.06, 59.82, 59.82, 59.82]
        koord_long = [10.73, 10.44, 10.44, 10.82, 10.92, 10.99]    
    
    map_osm = folium.Map(location=[koord_lat[0], koord_long[0]], zoom_start=zoomlevel, tiles='openstreetmap',zoom_control=False)     #Definerer plassering og zoomnivå på kartet
    
    mappe = f"Kart_GIF_Folium_{navn}"                                                                   #Definerer mappenavn og oppretter mappe
    current_directory = os.getcwd()
    final_directory = os.path.join(current_directory, mappe)
    if not os.path.exists(final_directory):                                                             # OBS: Lager ny mappe, men overskriver IKKE eksisterende mappe med samme navn
        os.makedirs(final_directory)

    antall = 0
    prosent = 0
    for AAR in range(startaar-1,sluttaar+1):                                                            
        borer_punkter = bronner_hvert_aar_borer(borer,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color='#1d3c34').add_to(map_osm), axis=1)
                                                                                                        # Bruker bronner_hvert_aar-funksjonen på hvert årstall
        antall_foer = antall
        antall = antall+len(borer_punkter)                                                              # Beregner antall og prosent til visning på figur
        if antall_foer>0:
            prosent = round((antall-antall_foer)/antall_foer*100)
        else:
            prosent = '-'

        folium.map.Marker(                                                                              # Legger til tittel-tekst på kartet
        [koord_lat[1], koord_long[1]],                                                                                        # Plassering (lengdegrad,breddegrad)
        icon=DivIcon(
            icon_size=(200,30),                                                                         # Størrelse på tekstboks
            icon_anchor=(0,0),
            #html='<div style="font-size: 20pt; font-family:Avenir, sans-serif; color:#1d3c34; background-color:#AAD3DF; text-align: left">'+'Energibrønner<br>registrert i NGU<br>GRANADA per<br>'+'</div>')
            html='<div style="font-size: 20pt; font-family:Avenir, sans-serif; color:#1d3c34; background-color:#F0EEE8; text-align: left">'+'Energibrønner<br>registrert i NGU<br>GRANADA per<br>'+'</div>')
        ).add_to(map_osm)
        
        folium.map.Marker(                                                                              # Legger til årstall på kartet
        [koord_lat[2], koord_long[2]],
        icon=DivIcon(
            icon_size=(120,30),
            icon_anchor=(0,0),
            #html='<div style="font-size: 40pt; font-family:Avenir, sans-serif; color:#1d3c34; background-color:#AAD3DF; text-align: left">'+str(AAR)+'</div>')
            html='<div style="font-size: 40pt; font-family:Avenir, sans-serif; color:#1d3c34; background-color:#F0EEE8; text-align: left">'+str(AAR)+'</div>')
        ).add_to(map_osm)

        folium.map.Marker([koord_lat[3], koord_long[3]],                                                                   # Legger til navn på oppdragstaker i legend
        icon=DivIcon(
            icon_size=(280,30),
            icon_anchor=(0,0),
            html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' Brønnborer:<br>'+figurnavn+'</div>'
            )
        ).add_to(map_osm)

        folium.map.Marker([koord_lat[4], koord_long[4]],                                                                   # Legger til antall i legend
        icon=DivIcon(
            icon_size=(70,30),
            icon_anchor=(0,0),
            html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">' 
            +'Antall:<br>'+str(antall)+'</div>'
            )
        ).add_to(map_osm)

        folium.map.Marker([koord_lat[5], koord_long[5]],                                                                   # Legger til prosentvis økning i legend
        icon=DivIcon(
            icon_size=(70,30),
            icon_anchor=(0,0),
            html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">' 
            +'Økning:<br>'+str(prosent)+' %</div>'
            )
        ).add_to(map_osm)
        
        map_osm.save(f"{mappe}/kart_{navn}_{AAR}.html")                                                 # Lagrer kartet som html-fil
        #webbrowser.open('Kart_'+borer+str(AAR)+'.html')                                                # Åpner html-filen i nettleser
        hti = Html2Image(custom_flags=['--virtual-time-budget=10000', '--hide-scrollbars'], output_path=mappe)          # Tar skjermbilde av utsnitt (900x900) av html-filen etter en tenkt tidsforsinkelse på 10 s og lagrer som png
        hti.screenshot(html_file=f"{mappe}/kart_{navn}_{AAR}.html", save_as=f"kart_{navn}_{AAR}.png", size=(900,900))

    frames = []                                                                                         # Setter sammen png-filer til GIF
    imgs = glob.glob(f"{mappe}/*.png")
    for i in imgs:
        new_frame = Image.open(i)
        frames.append(new_frame)

    frames[0].save(f"{mappe}/Energibronner_{navn}.gif", format='GIF',                                   # Lagrer gif-fil
                append_images=frames[1:],
                save_all=True,
                duration=750, loop=0)

Bruk av funksjonen for å generere GIF (Tar gjerne flere minutter per)

In [84]:
#bronner_fra_oppdragstaker(df,2022,2024,'Alle_oppdatert_2024','Alle','Norge')
#bronner_fra_oppdragstaker(df,1992,2024,'Alle_Oslo','Alle','Oslo')

#bronner_fra_oppdragstaker(baasum,1996,2023,'Båsum','Båsum Boring')
#bronner_fra_oppdragstaker(rototec,2012,2023,'Rototec','Rototec')
#bronner_fra_oppdragstaker(kraftenergibronn,2005,2023,'KraftEnergiBrønnboring','Kraft Energi & Brønnboring')
#bronner_fra_oppdragstaker(aarsleff,2002,2023,'Aarsleff','Aarsleff Boring')
#bronner_fra_oppdragstaker(seabrokers,1996,2023,'Seabrokers','Seabrokers GeoEnergi')
#bronner_fra_oppdragstaker(vestnorsk,1997,2023,'Vestnorsk','Vestnorsk Brunnboring')
#bronner_fra_oppdragstaker(universal,1996,2023,'Universal','Universal Brønnboring')
#bronner_fra_oppdragstaker(follo,2001,2023,'Follo','Follo Brønnboring')
#bronner_fra_oppdragstaker(vaeraas,1992,2023,'Værås','Værås Brønnboring')
#bronner_fra_oppdragstaker(brodrenemyhre,1993,2023,'BrødreneMyhre','Brødrene Myhre')

Genererer GIF med de 10 største oppdragstakerne med forskjellige farger (kan effektiviseres noe)

In [6]:


#Definerer input:
startaar=1996
sluttaar=2024
navn = 'Største_oppdatert_2024'
sted = 'Norge'  

#Farger som benyttes:
farger = ["#1d3c34", "#48a23f", "#ffa300", "#7c1158", "#4421af", "#1a53ff","#dc0ab4","#6cd4c5","#e14b31","#ede15b"]

if sted == 'Norge':
    zoomlevel = 5
    koord_lat = [65, 71, 69.3, 62.5, 62.5, 62.5, 62.5]
    koord_long = [18, 2, 2, 20.6, 21.5, 31.8, 34.3]
elif sted == 'Oslo':
    zoomlevel = 11
    koord_lat = [59.95, 60.10, 60.06, 59.82, 59.82, 59.82]              # Ikke ferdig!
    koord_long = [10.73, 10.44, 10.44, 10.82, 10.92, 10.99] 

#Definerer kart:
map_osm = folium.Map(location=[koord_lat[0], koord_long[0]], zoom_start=5, tiles='openstreetmap',zoom_control=False)

#Oppretter ny mappe:
mappe = f"Kart_GIF_Folium_{navn}"
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, mappe)
if not os.path.exists(final_directory):                         # OBS: Lager ny mappe, men overskriver IKKE eksisterende mappe med samme navn
    os.makedirs(final_directory)

#Definerer startverdier for antall og prosentvis økning:
antall_baasum = 0
antall_rototec = 0
antall_kraftenergibronn = 0
antall_aarsleff = 0
antall_seabrokers = 0
antall_vestnorsk = 0
antall_universal = 0
antall_follo = 0
antall_vaeraas = 0
antall_brodrenemyhre = 0
prosent_baasum = 0
prosent_rototec = 0
prosent_kraftenergibronn = 0
prosent_aarsleff = 0
prosent_seabrokers = 0
prosent_vestnorsk = 0
prosent_universal = 0
prosent_follo = 0
prosent_vaeraas = 0
prosent_brodrenemyhre = 0
prosent_andre = 0

#Kjører for hvert år:
for AAR in range(startaar-1,sluttaar+1):
    
    #Legger til punkter på kartet med forskjellige farger på forskjellige oppdragstakere:
    baasum_punkter = bronner_hvert_aar_borer(baasum,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[0]).add_to(map_osm), axis=1)
    rototec_punkter = bronner_hvert_aar_borer(rototec,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[1]).add_to(map_osm), axis=1)
    kraftenergibronn_punkter = bronner_hvert_aar_borer(kraftenergibronn,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[2]).add_to(map_osm), axis=1)
    aarsleff_punkter = bronner_hvert_aar_borer(aarsleff,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[3]).add_to(map_osm), axis=1)
    seabrokers_punkter = bronner_hvert_aar_borer(seabrokers,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[4]).add_to(map_osm), axis=1)
    vestnorsk_punkter = bronner_hvert_aar_borer(vestnorsk,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[5]).add_to(map_osm), axis=1)
    universal_punkter = bronner_hvert_aar_borer(universal,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[6]).add_to(map_osm), axis=1)
    follo_punkter = bronner_hvert_aar_borer(follo,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[7]).add_to(map_osm), axis=1)
    vaeraas_punkter = bronner_hvert_aar_borer(vaeraas,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[8]).add_to(map_osm), axis=1)
    brodrenemyhre_punkter = bronner_hvert_aar_borer(brodrenemyhre,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[9]).add_to(map_osm), axis=1)
    
    #Definerer variabel til prosentutregning:
    antall_baasum_foer = antall_baasum
    antall_rototec_foer = antall_rototec
    antall_kraftenergibronn_foer = antall_kraftenergibronn
    antall_aarsleff_foer = antall_aarsleff
    antall_seabrokers_foer = antall_seabrokers
    antall_vestnorsk_foer = antall_vestnorsk
    antall_universal_foer = antall_universal
    antall_follo_foer = antall_follo
    antall_vaeraas_foer = antall_vaeraas
    antall_brodrenemyhre_foer = antall_brodrenemyhre

    #Regner ut antall som skrives ut til legend:
    antall_baasum = antall_baasum+len(baasum_punkter)
    antall_rototec = antall_rototec+len(rototec_punkter)
    antall_kraftenergibronn = antall_kraftenergibronn+len(kraftenergibronn_punkter)
    antall_aarsleff = antall_aarsleff+len(aarsleff_punkter)
    antall_seabrokers = antall_seabrokers+len(seabrokers_punkter)
    antall_vestnorsk = antall_vestnorsk+len(vestnorsk_punkter)
    antall_universal = antall_universal+len(universal_punkter)
    antall_follo = antall_follo+len(follo_punkter)
    antall_vaeraas = antall_vaeraas+len(vaeraas_punkter)
    antall_brodrenemyhre = antall_brodrenemyhre+len(brodrenemyhre_punkter)

    #Regner ut prosentvis økning fra forrige år som skrives ut til legend. Dersom forrige års antall var 0, blir økningen "- %".
    if antall_baasum_foer>0:
        prosent_baasum = round((antall_baasum-antall_baasum_foer)/antall_baasum_foer*100)
    else:
        prosent_baasum = '-'
    if antall_rototec_foer>0:
        prosent_rototec = round((antall_rototec-antall_rototec_foer)/antall_rototec_foer*100)
    else:
        prosent_rototec = '-'
    if antall_kraftenergibronn_foer>0:
        prosent_kraftenergibronn = round((antall_kraftenergibronn-antall_kraftenergibronn_foer)/antall_kraftenergibronn_foer*100)
    else:
        prosent_kraftenergibronn = '-'
    if antall_aarsleff_foer>0:
        prosent_aarsleff= round((antall_aarsleff-antall_aarsleff_foer)/antall_aarsleff_foer*100)
    else:
        prosent_aarsleff = '-'
    if antall_seabrokers_foer>0:
        prosent_seabrokers = round((antall_seabrokers-antall_seabrokers_foer)/antall_seabrokers_foer*100)
    else:
        prosent_seabrokers = '-'
    if antall_vestnorsk_foer>0:
        prosent_vestnorsk = round((antall_vestnorsk-antall_vestnorsk_foer)/antall_vestnorsk_foer*100)
    else:
        prosent_vestnorsk = '-'
    if antall_universal_foer>0:
        prosent_universal = round((antall_universal-antall_universal_foer)/antall_universal_foer*100)
    else:
        prosent_universal = '-'
    if antall_follo_foer>0:
        prosent_follo = round((antall_follo-antall_follo_foer)/antall_follo_foer*100)
    else:
        prosent_follo = '-'
    if antall_vaeraas_foer>0:
        prosent_vaeraas = round((antall_vaeraas-antall_vaeraas_foer)/antall_vaeraas_foer*100)
    else:
        prosent_vaeraas = '-'
    if antall_brodrenemyhre_foer>0:
        prosent_brodrenemyhre = round((antall_brodrenemyhre-antall_brodrenemyhre_foer)/antall_brodrenemyhre_foer*100)
    else:
        prosent_brodrenemyhre = '-'


    #Legger til tittel-tekst:
    folium.map.Marker([koord_lat[1], koord_long[1]],
    icon=DivIcon(
        icon_size=(285,30),
        icon_anchor=(0,0),
        html='<div style="font-size: 20pt; font-family:Avenir, sans-serif; color:#1d3c34; background-color:#AAD3DF; text-align: left">'+'Energibrønner<br>registrert i NGU<br>GRANADA per<br>'+'</div>',
        )
    ).add_to(map_osm)
    
    #Legger til årstall på kartet:
    folium.map.Marker([koord_lat[2], koord_long[2]],
    icon=DivIcon(
        icon_size=(120,30),
        icon_anchor=(0,0),
        html='<div style="font-size: 40pt; font-family:Avenir, sans-serif; color:#1d3c34; background-color:#AAD3DF; text-align: left">'+str(AAR)+'</div>',
        )
    ).add_to(map_osm)

    #Legger til prikker i ulike farger som utgjør legend:
    folium.map.Marker([koord_lat[3], koord_long[3]],
    icon=DivIcon(
        icon_size=(50,30),
        icon_anchor=(0,0),
        html='<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' <br> &#11044'+
        '</div><div style="color:#48a23f; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#ffa300; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#7c1158; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#4421af; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#1a53ff; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#dc0ab4; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#6cd4c5; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#e14b31; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#ede15b; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div>'
        )
    ).add_to(map_osm)

    #Legger til navn på oppdragstakerne i legend:
    folium.map.Marker([koord_lat[4], koord_long[4]],
    icon=DivIcon(
        icon_size=(280,30),
        icon_anchor=(0,0),
        html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' Brønnborer:<br>'+
        'Båsum Boring:<br>Rototec:<br>Kraft Energi & Brønnboring:<br>Aarsleff Boring:<br>Seabrokers GeoEnergi:<br>Vestnorsk Brunnboring:<br>Universal Brønnboring:<br>Follo Brønnboring:<br>Værås Brønnboring:<br>Brødrene Myhre:'+
        '</div>'
        )
    ).add_to(map_osm)

    #Legger til antall brønner per oppdragstaker i legend:
    folium.map.Marker([koord_lat[5], koord_long[5]],
    icon=DivIcon(
        icon_size=(70,30),
        icon_anchor=(0,0),
        html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">' 
        +'Antall:<br>'+str(antall_baasum)+'<br>'+str(antall_rototec)+'<br>'+str(antall_kraftenergibronn)+'<br>'+str(antall_aarsleff)+'<br>'+str(antall_seabrokers)+'<br>'+str(antall_vestnorsk)+'<br>'+str(antall_universal)+'<br>'+str(antall_follo)+'<br>'+str(antall_vaeraas)+'<br>'+str(antall_brodrenemyhre)+
        '</div>'
        )
    ).add_to(map_osm)

    #Legger til prosentvis økning fra forrige år for hver oppdragstaker i legend:
    folium.map.Marker([koord_lat[6], koord_long[6]],
    icon=DivIcon(
        icon_size=(70,30),
        icon_anchor=(0,0),
        html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">' 
        +'Økning:<br>'+str(prosent_baasum)+' %<br>'+str(prosent_rototec)+' %<br>'+str(prosent_kraftenergibronn)+' %<br>'+str(prosent_aarsleff)+' %<br>'+str(prosent_seabrokers)+' %<br>'+str(prosent_vestnorsk)+' %<br>'+str(prosent_universal)+' %<br>'+str(prosent_follo)+' %<br>'+str(prosent_vaeraas)+' %<br>'+str(prosent_brodrenemyhre)+
        ' %</div>'
        )
    ).add_to(map_osm)

    #Lager kartet som html-fil i mappen
    map_osm.save(f"{mappe}/kart_{navn}_{AAR}.html")

    #Åpner html-filen i nettleser:
    #webbrowser.open('Kart_'+borer+str(AAR)+'.html')

    #Tar skjermbilde av html-filene og lagrer som png-filer i mappen:
    hti = Html2Image(custom_flags=['--virtual-time-budget=10000', '--hide-scrollbars'], output_path=mappe)
    hti.screenshot(html_file=f"{mappe}/kart_{navn}_{AAR}.html", save_as=f"kart_{navn}_{AAR}.png", size=(900,900))

#Setter sammen png-filene til en gif og lagrer i mappen:
frames = []
imgs = glob.glob(f"{mappe}/*.png")
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
frames[0].save(f"{mappe}/Energibronner_{navn}.gif", format='GIF',
            append_images=frames[1:],
            save_all=True,
            duration=750, loop=0)

'''

'\n\n#Definerer input:\nstartaar=1996\nsluttaar=2023\nnavn = \'Største\'\n\n#Farger som benyttes:\nfarger = ["#1d3c34", "#48a23f", "#ffa300", "#7c1158", "#4421af", "#1a53ff","#dc0ab4","#6cd4c5","#e14b31","#ede15b"]\n\n#Definerer kart:\nmap_osm = folium.Map(location=[65, 18], zoom_start=5, tiles=\'openstreetmap\',zoom_control=False)\n\n#Oppretter ny mappe:\nmappe = f"Kart_GIF_Folium_{navn}"\ncurrent_directory = os.getcwd()\nfinal_directory = os.path.join(current_directory, mappe)\nif not os.path.exists(final_directory):                         # OBS: Lager ny mappe, men overskriver IKKE eksisterende mappe med samme navn\n    os.makedirs(final_directory)\n\n#Definerer startverdier for antall og prosentvis økning:\nantall_baasum = 0\nantall_rototec = 0\nantall_kraftenergibronn = 0\nantall_aarsleff = 0\nantall_seabrokers = 0\nantall_vestnorsk = 0\nantall_universal = 0\nantall_follo = 0\nantall_vaeraas = 0\nantall_brodrenemyhre = 0\nprosent_baasum = 0\nprosent_rototec = 0\nprosent_kraftener

Genererer GIF med alle brønner i forskjellige farger etter type (kan effektiviseres noe)

In [10]:
#Definerer input:
startaar=1992
sluttaar=2023
navn = 'Alle_etter_type'

#Farger som benyttes:
farger = ["#1d3c34", "#48a23f", "#ffa300", "#7c1158", "#4421af", "#1a53ff","#dc0ab4","#6cd4c5","#e14b31","#ede15b"]

#Definerer kart:
map_osm = folium.Map(location=[65, 18], zoom_start=5, tiles='openstreetmap',zoom_control=False)

#Oppretter ny mappe:
mappe = f"Kart_GIF_Folium_{navn}"
current_directory = os.getcwd()
final_directory = os.path.join(current_directory, mappe)
if not os.path.exists(final_directory):                         # OBS: Lager ny mappe, men overskriver IKKE eksisterende mappe med samme navn
    os.makedirs(final_directory)

#Definerer startverdier for antall og prosentvis økning:
antall_enkelthusholdning = 0
antall_storre_anlegg = 0
antall_andre_anlegg = 0

prosent_enkelthusholdning = 0
prosent_storre_anlegg = 0
prosent_andre_anlegg = 0

#Kjører for hvert år:
for AAR in range(startaar-1,sluttaar+1):
    
    #Legger til punkter på kartet med forskjellige farger på forskjellige oppdragstakere:
    andre_anlegg_punkter = bronner_hvert_aar_borer(andre_anlegg,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[2]).add_to(map_osm), axis=1)
    storre_anlegg_punkter = bronner_hvert_aar_borer(storre_anlegg,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[1]).add_to(map_osm), axis=1)
    enkelthusholdning_punkter = bronner_hvert_aar_borer(enkelthusholdning,AAR).apply(lambda row:folium.CircleMarker(location=[row["y_koordinat"], row["x_koordinat"]],radius=0.01,color=farger[0]).add_to(map_osm), axis=1)
    
    #Definerer variabel til prosentutregning:
    antall_enkelthusholdning_foer = antall_enkelthusholdning
    antall_storre_anlegg_foer = antall_storre_anlegg
    antall_andre_anlegg_foer = antall_andre_anlegg

    #Regner ut antall som skrives ut til legend:
    antall_enkelthusholdning = antall_enkelthusholdning+len(enkelthusholdning_punkter)
    antall_storre_anlegg = antall_storre_anlegg+len(storre_anlegg_punkter)
    antall_andre_anlegg = antall_andre_anlegg+len(andre_anlegg_punkter)

    #Regner ut prosentvis økning fra forrige år som skrives ut til legend. Dersom forrige års antall var 0, blir økningen "- %".
    if antall_enkelthusholdning_foer>0:
        prosent_enkelthusholdning = round((antall_enkelthusholdning-antall_enkelthusholdning_foer)/antall_enkelthusholdning_foer*100)
    else:
        prosent_enkelthusholdning = '-'
    if antall_storre_anlegg_foer>0:
        prosent_storre_anlegg = round((antall_storre_anlegg-antall_storre_anlegg_foer)/antall_storre_anlegg_foer*100)
    else:
        prosent_storre_anlegg = '-'
    if antall_andre_anlegg_foer>0:
        prosent_andre_anlegg = round((antall_andre_anlegg-antall_andre_anlegg_foer)/antall_andre_anlegg_foer*100)
    else:
        prosent_andre_anlegg = '-'

    #Legger til tittel-tekst:
    folium.map.Marker([71, 2],
    icon=DivIcon(
        icon_size=(285,30),
        icon_anchor=(0,0),
        html='<div style="font-size: 20pt; font-family:Avenir, sans-serif; color:#1d3c34; background-color:#AAD3DF; text-align: left">'+'Energibrønner<br>registrert i NGU<br>GRANADA per<br>'+'</div>',
        )
    ).add_to(map_osm)
    
    #Legger til årstall på kartet:
    folium.map.Marker([69.3, 2],
    icon=DivIcon(
        icon_size=(120,30),
        icon_anchor=(0,0),
        html='<div style="font-size: 40pt; font-family:Avenir, sans-serif; color:#1d3c34; background-color:#AAD3DF; text-align: left">'+str(AAR)+'</div>',
        )
    ).add_to(map_osm)

    #Legger til prikker i ulike farger som utgjør legend:
    folium.map.Marker([58.5, 20.6],
    icon=DivIcon(
        icon_size=(50,30),
        icon_anchor=(0,0),
        html='<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' <br> &#11044'+
        '</div><div style="color:#48a23f; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div><div style="color:#ffa300; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' &#11044'+
        '</div>'
        )
    ).add_to(map_osm)

    #Legger til navn på oppdragstakerne i legend:
    folium.map.Marker([58.5, 21.5],
    icon=DivIcon(
        icon_size=(280,30),
        icon_anchor=(0,0),
        html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">'+' Type anlegg:<br>'+
        'Enkelthusholdning:<br>Større anlegg:<br>Andre'+
        '</div>'
        )
    ).add_to(map_osm)

    #Legger til antall brønner per oppdragstaker i legend:
    folium.map.Marker([58.5, 31.8],
    icon=DivIcon(
        icon_size=(70,30),
        icon_anchor=(0,0),
        html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">' 
        +'Antall:<br>'+str(antall_enkelthusholdning)+'<br>'+str(antall_storre_anlegg)+'<br>'+str(antall_andre_anlegg)+
        '</div>'
        )
    ).add_to(map_osm)

    #Legger til prosentvis økning fra forrige år for hver oppdragstaker i legend:
    folium.map.Marker([58.5, 34.3],
    icon=DivIcon(
        icon_size=(70,30),
        icon_anchor=(0,0),
        html=f'<div style="color:#1d3c34; font-size: 14pt; font-family:Avenir, sans-serif; background-color:#F0EEE8; text-align: left">' 
        +'Økning:<br>'+str(prosent_enkelthusholdning)+' %<br>'+str(prosent_storre_anlegg)+' %<br>'+str(prosent_andre_anlegg)+
        ' %</div>'
        )
    ).add_to(map_osm)

    #Lager kartet som html-fil i mappen
    map_osm.save(f"{mappe}/kart_{navn}_{AAR}.html")

    #Åpner html-filen i nettleser:
    #webbrowser.open('Kart_'+borer+str(AAR)+'.html')

    #Tar skjermbilde av html-filene og lagrer som png-filer i mappen:
    hti = Html2Image(custom_flags=['--virtual-time-budget=10000', '--hide-scrollbars'], output_path=mappe)
    hti.screenshot(html_file=f"{mappe}/kart_{navn}_{AAR}.html", save_as=f"kart_{navn}_{AAR}.png", size=(900,900))

#Setter sammen png-filene til en gif og lagrer i mappen:
frames = []
imgs = glob.glob(f"{mappe}/*.png")
for i in imgs:
    new_frame = Image.open(i)
    frames.append(new_frame)
frames[0].save(f"{mappe}/Energibronner_{navn}.gif", format='GIF',
            append_images=frames[1:],
            save_all=True,
            duration=750, loop=0)

In [6]:
map_osm = folium.Map(location=[59.93, 10.67], zoom_start=12, tiles='openstreetmap',zoom_control=False)
map_osm